In [1]:
import time
import random
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# from Feed import Feed
from typing import Iterator, Any

In [2]:
from Surfer import GetBrowser,RedditFeed,Feed

driver = GetBrowser()

[WDM] - Downloading: 100%|██████████| 6.80M/6.80M [00:14<00:00, 500kB/s]


In [58]:
def get_script(fname):
  script = ''
  with open(f'./scripts/{fname}.js','r') as f:
    script = f.read()
  script = script.replace('\n','')
  def replacer(*args):
    acc = script
    for i,x in enumerate(args):
      acc = acc.replace('#'+str(i),x)
    return acc
  return replacer

In [59]:
get_script('init_reddit')('h3')

"let x = [].slice.call(document.querySelectorAll(`div[data-scroller-first]`));x = x.filter(x => x.querySelector('i.icon-upvote'))[0] || x[0];x = x.querySelector(`h3`);console.log(x);return x"

In [20]:
class RedditFeed(Feed):
	def __init__(self, driver):
		super()
		self.name = 'Reddit Feed'
		# Store the WebDriver instance
		self.driver = driver
		self.driver.get('https://www.reddit.com/')

		# Find the div[data-scroller-first] element
		self.first_element = WebDriverWait(self.driver, 10).until(
			EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-scroller-first]'))
		)
		# Select all div elements with the attribute 'data-scroller-first'
		divs = driver.find_elements(By.CSS_SELECTOR,"div[data-scroller-first]")

		# Filter the div elements that have a child element 'i' with class 'icon-upvote'
		filtered_divs = [div for div in divs if div.find_elements(By.CSS_SELECTOR,"i.icon-upvote")]

		# Perform actions on the filtered div elements
		for filtered_div in filtered_divs:
				# Perform actions on the filtered div element
				# For example, you can retrieve text, click, or interact with the element as needed
				print(filtered_div.text)
		self.the_first_element = divs[0]
		self.first_element = self.the_first_element

		print(self.the_first_element)

	def __iter__(self) -> Iterator[Any]:
	  # Check if there are no more siblings
		if not self.first_element:
			raise StopIteration

		# Scroll the first element into view with random scroll position
		self.scroll_element_into_view(self.first_element, random.uniform(0.2, 0.8))

		# Get the parent element of the first_element
		parent_element = self.first_element.find_element(By.XPATH, '..')

		# Get all the child elements (siblings) of the parent_element
		siblings = parent_element.find_elements(By.XPATH, '*')

		# Iterate over the siblings and yield their text
		for sibling in siblings:
			# Scroll each sibling element into view with random scroll position
			self.scroll_element_into_view(sibling, random.uniform(0.2, 0.8))  # Add random scroll position between 0.2 and 0.8
			yield sibling.text

		# Set the first_element to None to mark the end of iteration
		self.first_element = None

	def scroll_element_into_view(self, element, scroll_position=0.5):
		# Get the element's height
		element_height = element.size['height']

		# Calculate the target scroll position based on the element's height and the given scroll position
		target_scroll_position = element.location['y'] + element_height * scroll_position

		# Scroll the page to the target scroll position using JavaScript
		self.driver.execute_script("window.scrollTo(0, arguments[0]);", target_scroll_position)

	def close(self):
		# Close the WebDriver instance
		self.driver.quit()

In [3]:
name = 'Reddit Feed'
# Store the WebDriver instance
driver = driver
driver.get('https://www.reddit.com/')

# Find the div[data-scroller-first] element
first_element = WebDriverWait(driver, 10).until(
	EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-scroller-first]'))
)

In [90]:
def run_script(name,verbose=False):
	def script(*args):
		script_string = get_script(name)(*args)
		if verbose:
			print('script_string: ',script_string)
		return driver.execute_script(script_string)
	return script
x = run_script('init_reddit')('h3')

print(x)

I can’t with the smugness…post has 10k comments and all of them supporting this like “yass go baby mama”.


In [85]:
run_script('reddit next')()

'12.9kr/facepalm•Posted byu/SyloriaRocius10 hours agoJoinI can’t with the smugness…post has 10k comments and all of them supporting this like “yass go baby mama”. 🇲\u200b🇮\u200b🇸\u200b🇨\u200b\n              \n            \n        .t3_12qbxb0._2FCtq-QzlfuN-SwVMUZMM3 {\n          --postTitle-VisitedLinkColor: #edeeef;\n          --postTitleLink-VisitedLinkColor: #6f7071;\n          --postBodyLink-VisitedLinkColor: #6f7071;\n        }\n      12.9k4.9k commentssharesave'

In [100]:
from time import sleep
for i in range(10):
	sleep(3)
	run_script('reddit scroll next',verbose=True)("0.5",'0.5')